## What is RAG ?

RAG is a technique where a language model first looks up (retrieves) useful information from a database or documents, and then uses that information to give a better answer.

## Why and when we prefer RAG over finetuning ?

We **prefer RAG over finetuning** when: We want the model to give **up-to-date or specific answers** from **our own data** without changing the model itself.

---

**Why prefer RAG?**

* **Cheaper & faster** – No need to train the model again.
* **Easier to update** – Just change the documents, not the model.
* **Better for private or large data** – You keep data separate and safe.

---

**When to use RAG?**

* When your data **changes often** (like news, product lists).
* When you want the model to **answer from your documents**.
* When **training a model is too costly or slow**.

---

Think of RAG like **giving the model a library to read** instead of teaching it everything from scratch.


# Install Libraries

* **`langchain`** – Core framework to build LLM-powered applications.
* **`langchain-community`** – Extra integrations like tools, APIs, and vector stores.
* **`langchain-pinecone`** – Connects LangChain with Pinecone for vector storage and retrieval.
* **`langchain_groq`** – Enables LangChain to use Groq's ultra-fast language models.
* **`datasets`** – Provides ready-to-use NLP/ML datasets from Hugging Face.

In [1]:
%pip install langchain==0.3.23 langchain-community==0.3.21 langchain-pinecone==0.2.5 langchain_groq datasets==3.5.0

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.4 MB/s  0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 26.5 MB/s  0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ---------------------------------------- 0.0/564.3 kB ? eta -:--:--
   ---------------------------------------- 564.3/564.3 kB 15.3 MB/s  0:00:00

   ----------------------------------------  0/26 [py-cpuinfo]
   --- ------------------------------------  2/26 [pinecone-plugin-interface]
   ---- -----------------------------------  3/26 [mdurl]
  Attempting uninstall: fsspec
   ---- -----------------------------------  3/26 [mdurl]
    Found existing installation: fsspec 2025.9.0
   ---- -----------------------------------  3/26 [mdurl]
    Uninstalling fsspec-2025.9.0:
   ---- -------------

## Load API Keys from .env file

### What is env file?

* A .env file is a simple text file that stores environment variables (like API keys and secrets) in key=value format.
* Example content of a .env file:
* PINECONE_API_KEY=your_pinecone_api_key_here
* GROQ_API_KEY=your_groq_api_key_here

* It helps keep sensitive information out of your code and makes it easier to manage secrets securely.

**Imports tools** to:

* Use environment variables (`os`)
* Load values from a `.env` file (`load_dotenv`)
* **os** is a Python built-in module that lets your code interact with the operating system (like Windows, macOS, Linux).
---
* **Loads the `.env` file** so Python can use the secret keys stored in it (like API keys).
* **Gets the values** of `PINECONE_API_KEY` and `GROQ_API_KEY` from the `.env` file.
* **In Short:** This code **reads your secret keys from a `.env` file** so you don’t have to write them directly in your code.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the keys
pinecone_key = os.getenv("PINECONE_API_KEY")
groq_key = os.getenv("GROQ_API_KEY")

# Check if keys are loaded properly
if pinecone_key:
    print("✅ Pinecone API Key Loaded Successfully")
else:
    print("❌ Pinecone API Key NOT Loaded")

if groq_key:
    print("✅ Groq API Key Loaded Successfully")
else:
    print("❌ Groq API Key NOT Loaded")


✅ Pinecone API Key Loaded Successfully
✅ Groq API Key Loaded Successfully


## What is `langchain_groq`?

`langchain_groq` is a **LangChain integration** that lets you **connect to Groq’s LLMs** (like LLaMA3) easily.

Think of it as a **bridge between LangChain and Groq’s fast language models**.

---

## What is `ChatGroq`?

`ChatGroq` is a **class (tool)** inside `langchain_groq`.

It lets you:

* **Send prompts** to Groq-hosted models
* **Receive responses** from those models
* Use these models in your **LangChain app**, like chatbots, RAG, agents, etc.

---

**Why do we use this?**

Instead of manually setting up HTTP requests to Groq’s API, `ChatGroq` makes it **super easy**:

* Lets you talk to a specific Groq model (`llama3-8b-8192`)
* Works smoothly with LangChain tools (retrievers, chains, memory, etc.)
* Connects securely with your `groq_api_key`

---

#### In Simple Words

* `langchain_groq` lets LangChain talk to Groq.
* `ChatGroq` is the tool that helps you **chat with Groq’s AI model** using your API key.


In [5]:
from langchain_groq import ChatGroq

chat = ChatGroq(
    groq_api_key=groq_key,
    model_name="Llama-3.3-70B-Versatile"
)

Groq uses the **same chat structure as OpenAI** because it runs **OpenAI-compatible models** like `llama3`, `mixtral`, etc.
So just like OpenAI, chats with Groq **typically look like this in plain text**:

```
System: You are a helpful assistant.
User: Hi, how are you?
Assistant: I'm doing well! How can I assist you today?
User: What is quantum computing?
Assistant:
```

The final `"Assistant:"` without a response is what would prompt the model to continue the conversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

---

**In Code (OpenAI/Groq-compatible format):**

When using the API (like with `ChatGroq` or `ChatOpenAI` in LangChain), you use this structure:

```
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "I'm doing well! How can I assist you today?"},
    {"role": "user", "content": "What is quantum computing?"}
]
```

---

**In LangChain (message objects):**

LangChain wraps those into **message classes**, like:

```
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi, how are you?"),
    AIMessage(content="I'm doing well! How can I assist you today?"),
    HumanMessage(content="What is quantum computing?")
]
```

The format is very similar, we're just swapped the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

Then you pass them to the model:

```
response = chat.invoke(messages)
print(response.content)
```

In [6]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

We generate the next response from the AI by passing these messages to the `ChatGroq` object.

Like saying to the AI:

“Here’s what has been said so far — now tell me what the AI should say next.”

LangChain then handles formatting and sending this to the LLM backend, and res stores the AI’s next reply.

**In Short:**
* You define a conversation (via messages).
* Call the LLM using chat(messages).
* Get a response back — stored in res.

In [7]:
res = chat(messages)

C:\Users\Anas\AppData\Local\Temp\ipykernel_17860\1671545441.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = chat(messages)


In [8]:
res

AIMessage(content='String theory is a complex and abstract concept in physics, but I\'ll try to break it down in simple terms.\n\n**What is string theory?**\n\nString theory is a theoretical framework in physics that attempts to reconcile two major areas of physics: general relativity (which describes gravity and the large-scale structure of the universe) and quantum mechanics (which describes the behavior of particles at the atomic and subatomic level).\n\nThe core idea of string theory is that the fundamental building blocks of the universe are not particles (like electrons and photons), but tiny, vibrating strings. These strings are thought to be the source of all particles and forces in the universe.\n\n**Key concepts:**\n\n1. **Strings**: The fundamental objects in string theory are one-dimensional strings, rather than point-like particles. These strings can vibrate at different frequencies, giving rise to various particles.\n2. **Vibrational modes**: The vibrational modes of the 

To see the models reply

In [9]:
print(res.content)

String theory is a complex and abstract concept in physics, but I'll try to break it down in simple terms.

**What is string theory?**

String theory is a theoretical framework in physics that attempts to reconcile two major areas of physics: general relativity (which describes gravity and the large-scale structure of the universe) and quantum mechanics (which describes the behavior of particles at the atomic and subatomic level).

The core idea of string theory is that the fundamental building blocks of the universe are not particles (like electrons and photons), but tiny, vibrating strings. These strings are thought to be the source of all particles and forces in the universe.

**Key concepts:**

1. **Strings**: The fundamental objects in string theory are one-dimensional strings, rather than point-like particles. These strings can vibrate at different frequencies, giving rise to various particles.
2. **Vibrational modes**: The vibrational modes of the strings correspond to different

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [10]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory, also known as a "theory of everything" (ToE), for several reasons:

1. **Unification of forces**: String theory attempts to unify the fundamental forces of nature, including:
	* Gravity (described by general relativity)
	* Electromagnetism (described by quantum electrodynamics)
	* Strong nuclear force (described by quantum chromodynamics)
	* Weak nuclear force (described by the electroweak theory)
	* String theory posits that these forces are different vibrational modes of the same underlying string.
2. **Consistency with quantum mechanics and general relativity**: String theory is an attempt to reconcile the principles of quantum mechanics (which describe the behavior of particles at the atomic and subatomic level) with the principles of general relativity (which describe gravity and the large-scale structure of the universe). By positing that particles are vibrations of strings, string theory aims to

## Dealing with Hallucinations

We have our chatbot, but as mentioned — the knowledge of LLMs can be limited. The reason for this is that LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model. We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world.

The result of this is very clear when we ask LLMs about more recent information, like about Deepseek R1.

In [11]:
print(res.content)

Physicists believe that string theory has the potential to produce a unified theory, also known as a "theory of everything" (ToE), for several reasons:

1. **Unification of forces**: String theory attempts to unify the fundamental forces of nature, including:
	* Gravity (described by general relativity)
	* Electromagnetism (described by quantum electrodynamics)
	* Strong nuclear force (described by quantum chromodynamics)
	* Weak nuclear force (described by the electroweak theory)
	* String theory posits that these forces are different vibrational modes of the same underlying string.
2. **Consistency with quantum mechanics and general relativity**: String theory is an attempt to reconcile the principles of quantum mechanics (which describe the behavior of particles at the atomic and subatomic level) with the principles of general relativity (which describe gravity and the large-scale structure of the universe). By positing that particles are vibrations of strings, string theory aims to

Our chatbot can no longer help us, it doesn't contain the information we need to answer the question. It was very clear from this answer that the LLM doesn't know the informaiton, but sometimes an LLM may respond like it _does_ know the answer — and this can be very hard to detect.

## Alternate Way : Source Knowledge

There is another way of feeding knowledge into LLMs. It is called _source knowledge_ and it refers to any information fed into the LLM via the prompt. We can try that with the Deepseek question. We can take the paper abstract from the [Deepseek R1 paper](https://arxiv.org/abs/2501.12948).

In [12]:
source_knowledge = (
    "We introduce our first-generation reasoning models, DeepSeek-R1-Zero and "
    "DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale "
    "reinforcement learning (RL) without supervised fine-tuning (SFT) as a "
    "preliminary step, demonstrates remarkable reasoning capabilities. Through "
    "RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and "
    "intriguing reasoning behaviors. However, it encounters challenges such as "
    "poor readability, and language mixing. To address these issues and "
    "further enhance reasoning performance, we introduce DeepSeek-R1, which "
    "incorporates multi-stage training and cold-start data before RL. "
    "DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on "
    "reasoning tasks. To support the research community, we open-source "
    "DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B, 8B, 14B, "
    "32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama."
)

We can feed this additional knowledge into our prompt with some instructions telling the LLM how we'd like it to use this information alongside our original query.

In [13]:
query = "What is so special about Deepseek R1?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

Now we feed this into our chatbot as we were before.

In [14]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [15]:
print(res.content)

According to the context, DeepSeek-R1 is special because it achieves performance comparable to OpenAI-o1-1217 on reasoning tasks, thanks to its multi-stage training and incorporation of cold-start data before reinforcement learning (RL). This suggests that DeepSeek-R1 has made significant advancements in reasoning capabilities, making it a notable model in the field. Additionally, the fact that its predecessor, DeepSeek-R1-Zero, was able to demonstrate remarkable reasoning capabilities without supervised fine-tuning (SFT) is also a notable achievement, and DeepSeek-R1 builds upon this foundation to further enhance its performance.


## How do we get this information in the first place?

The quality of this answer is phenomenal. This is made possible thanks to the idea of augmented our query with external knowledge (source knowledge). There's just one problem

This is where Pinecone and vector databases comes in place, as they can help us here too. But first, we'll need a dataset.

## Importing the Data

In this task, we will be importing our data. We will be using the Hugging Face Datasets library to load our data. Specifically, we will be using the `"jamescalam/deepseek-r1-paper-chunked"` dataset. This dataset contains the Deepseek R1 paper pre-processed into RAG-ready chunks.

In [16]:
from datasets import load_dataset

dataset = load_dataset(
    "jamescalam/deepseek-r1-paper-chunked",
    split="train"
)

dataset

C:\Users\Anas\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Anas\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anas\.cache\huggingface\hub\datasets--jamescalam--deepseek-r1-paper-chunked. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'num_tokens', 'pages', 'source'],
    num_rows: 76
})

In [17]:
dataset[0]

{'doi': '2501.12948v1',
 'chunk-id': 1,
 'chunk': "uestion: If a > 1, then the sum of the real solutions of √a - √a + x = x is equal to Response: <think> To solve the equation √a – √a + x = x, let's start by squaring both . . . (√a-√a+x)² = x² ⇒ a - √a + x = x². Rearrange to isolate the inner square root term:(a – x²)² = a + x ⇒ a² – 2ax² + (x²)² = a + x ⇒ x⁴ - 2ax² - x + (a² – a) = 0",
 'num_tokens': 145,
 'pages': [1],
 'source': 'https://arxiv.org/abs/2501.12948'}

## Dataset Overview

The dataset we are using is sourced from the Deepseek R1 ArXiv papers. Each entry in the dataset represents a "chunk" of text from the R1 paper.

Because most **L**arge **L**anguage **M**odels (LLMs) only contain knowledge of the world as it was during training, even many of the newest LLMs cannot answer questions about Deepseek R1 — at least not without this data.

## Building the Knowledge Base

We now have a dataset that can serve as our chatbot knowledge base. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

We begin by initializing our Pinecone client, this requires a [free API key](https://app.pinecone.io).

In [18]:
from pinecone import Pinecone

# initialize client
pc = Pinecone(api_key=pinecone_key)

Delete the old one to save the resources

In [34]:
index_name = "rag1"
pc.delete_index(index_name)

In [35]:
from pinecone import ServerlessSpec, CloudProvider, AwsRegion, Metric

pc.create_index(
    name=index_name,
    metric=Metric.DOTPRODUCT,
    dimension=384,  # ✅ match your embedding model
    spec=ServerlessSpec(
        cloud=CloudProvider.AWS,
        region=AwsRegion.US_EAST_1
    )
)
index = pc.Index(index_name)

Our index is now ready but it's empty. It is a vector index, so it needs vectors. As mentioned, to create these vector embeddings we will HuggingFace's `sentence-transformers/all-MiniLM-L6-v2` model — we can access it via LangChain like so:

In [24]:
pip install sentence-transformers

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.0 MB 6.7 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/12.0 MB 8.2 MB/s eta 0:00:02
   -------------------- ------------------- 6.0/12.0 MB 10.4 MB/s eta 0:00:01
   ---------------------------------- ----- 10.5/12.0 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 13.6 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 24.3 MB/s  0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -------------- ------------------------- 3.1/8.7 MB 15.3 MB/s eta 0:00:01
   ---------------- ----------------------- 3.7/8.7 MB 10.3 MB/s eta 0:00:01
   ------------------------ --------------- 5.2/8.7 MB 8.2 MB/s eta 0:00:01
   ------------------------------ --------- 6.6/8.7 MB 7.7 MB/s eta 0:00:01
   ----------------------

In [36]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Using this model we can create embeddings like so:

In [37]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 384)

From this we get two (aligning to our two chunks of text) CHANGE-dimensional embeddings.

We're now ready to embed and index all our our data! We do this by looping through our dataset and embedding and inserting everything in batches.

In [38]:
from tqdm.auto import tqdm  # for progress bar

data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.22s/it]


We can check that the vector index has been populated using `describe_index_stats` like before:

In [39]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# Retrieval Augmented Generation

We've built a fully-fledged knowledge base. Now it's time to link that knowledge base to our chatbot. To do that we'll be diving back into LangChain and reusing our template prompt from earlier.

To use LangChain here we need to load the LangChain abstraction for a vector index, called a `vectorstore`. We pass in our vector `index` to initialize the object.

In [40]:
from langchain_pinecone import PineconeVectorStore

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embed_model,
    text_key=text_field
)

Using this `vectorstore` we can already query the index and see if we have any relevant information given our question about Llama 2.

In [41]:
query = "What is so special about Deepseek R1?"

vectorstore.similarity_search(query, k=3)

[Document(id='2501.12948v1-39', metadata={'source': 'https://arxiv.org/abs/2501.12948'}, page_content='## 1.2. Summary of Evaluation Results - **Reasoning tasks:** (1) DeepSeek-R1 achieves a score of 79.8% Pass@1 on AIME 2024, slightly surpassing OpenAI-01-1217. On MATH-500, it attains an impressive score of 97.3%, performing on par with OpenAI-01-1217 and significantly outperforming other models. (2) On coding-related tasks, DeepSeek-R1 demonstrates expert level in code competition tasks, as it achieves 2,029 Elo rating on Codeforces outperforming 96.3% human participants in the competition. For engineering-related tasks, DeepSeek-R1 performs slightly better than DeepSeek-V3, which could help developers in real world tasks. - **Knowledge:** On benchmarks such as MMLU, MMLU-Pro, and GPQA Diamond, DeepSeek-R1 achieves outstanding results, significantly outperforming DeepSeek-V3 with scores of 90.8% on MMLU, 84.0% on MMLU-Pro, and 71.5% on GPQA Diamond. While its performance is slightly 

We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All we need is to link the output from our `vectorstore` to our `chat` chatbot. To do that we can use the same logic as we used earlier.

In [42]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [43]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    ## 1.2. Summary of Evaluation Results - **Reasoning tasks:** (1) DeepSeek-R1 achieves a score of 79.8% Pass@1 on AIME 2024, slightly surpassing OpenAI-01-1217. On MATH-500, it attains an impressive score of 97.3%, performing on par with OpenAI-01-1217 and significantly outperforming other models. (2) On coding-related tasks, DeepSeek-R1 demonstrates expert level in code competition tasks, as it achieves 2,029 Elo rating on Codeforces outperforming 96.3% human participants in the competition. For engineering-related tasks, DeepSeek-R1 performs slightly better than DeepSeek-V3, which could help developers in real world tasks. - **Knowledge:** On benchmarks such as MMLU, MMLU-Pro, and GPQA Diamond, DeepSeek-R1 achieves outstanding results, significantly outperforming DeepSeek-V3 with scores of 90.8% on MMLU, 84.0% on MMLU-Pro, and 71.5% on GPQA Diamond. While its performance is slightly below that of OpenAI-01-1217 on these be

There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

In [44]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

DeepSeek-R1 is special because it achieves competitive performance in various reasoning tasks, often surpassing or matching the performance of other leading models like OpenAI-01-1217. Some notable aspects of DeepSeek-R1 include:

1. **High scores on reasoning benchmarks**: DeepSeek-R1 achieves impressive scores on benchmarks like AIME 2024 (79.8% Pass@1), MATH-500 (97.3%), and GPQA Diamond (71.5%).
2. **Expert-level performance on coding tasks**: DeepSeek-R1 demonstrates expert-level performance on coding-related tasks, with an Elo rating of 2029 on Codeforces, outperforming 96.3% of human participants.
3. **Competitive edge in educational tasks**: DeepSeek-R1 performs well on knowledge-related benchmarks like MMLU, MMLU-Pro, and GPQA Diamond, making it a strong contender for educational applications.
4. **Robust performance across distilled models**: The distilled models of DeepSeek-R1, such as DeepSeek-R1-Distill-Qwen-32B and DeepSeek-R1-Distill-Llama-70B, consistently rank high acr

We can continue with another Deepseek R1:

In [45]:
prompt = HumanMessage(
    content=augment_prompt(
        "how does deepseek r1 compare to deepseek r1 zero?"
    )
)

res = chat(messages + [prompt])
print(res.content)

The contexts provided do not offer a direct comparison between DeepSeek-R1 and DeepSeek-R1-Zero. However, it is mentioned that DeepSeek-R1 was introduced to address the challenges faced by DeepSeek-R1-Zero, such as poor readability and language mixing.

DeepSeek-R1-Zero is described as a model trained via large-scale reinforcement learning (RL) without supervised fine-tuning (SFT) as a preliminary step, which demonstrates remarkable reasoning capabilities. On the other hand, DeepSeek-R1 incorporates multi-stage training and cold-start data before RL, which helps to enhance its reasoning performance.

While the exact comparison between DeepSeek-R1 and DeepSeek-R1-Zero is not provided, it can be inferred that DeepSeek-R1 is an improved version of DeepSeek-R1-Zero, with better performance and addressing some of the challenges faced by its predecessor. However, the specific details of the comparison are not available in the given contexts.


You can continue asking questions about Deepseek R1, but once you're done you can delete the index to save resources:

In [46]:
pc.delete_index(index_name)